#### Fulfillment of Coursera projects

<h2> <u> Table of Contents </u> </h2>

[1. Question 1](#Question-1)  
[2. Question 2](#Question-2)   
[3. Question 3](#Question-3)

#### Question 1

In [1]:
!pip install bs4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
html = requests.get(url) #getting html of the webpage

soup = BeautifulSoup(html.text) ## converting the html to soup element

In [5]:
table = soup.find_all('table', class_='wikitable sortable') ##finding table we need 

In [6]:
header = [x.text.replace('\n', '') for x in table[0].tr.find_all('th')]  # finding header
header

['Postal Code', 'Borough', 'Neighborhood']

In [7]:
#creating pandas data frame 
df = pd.DataFrame(columns=header)
data = {}
for tr in table[0].find_all('tr')[1:]:  #looping through 2 row to end
    td = [x.text.replace('\n', '') for x in tr.find_all('td')] #looping through all the table data in table row
    data['Postal Code'] = td[0]
    data['Borough'] = td[1]
    data['Neighborhood'] = td[2]
    df = df.append(data, ignore_index=True) #appending new row to existing data frame


In [8]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']  #renameing the columns

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#removing columns with Borough value "Not assigned"
df_clean = df[df.Borough != 'Not assigned'] 

In [11]:
df_clean[list(df_clean.groupby(['PostalCode'])['Borough'].count() != 1)]  #no duplicate value of Postal Code

,PostalCode,Borough,Neighborhood


In [12]:
df_clean.reset_index(drop=True,inplace=True)

In [13]:
df_clean.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
print(f' There are {df_clean.shape[0]} rows in the data frame')

 There are 103 rows in the data frame


#### Question 2

In [15]:
!pip install geocoder

In [16]:
import geocoder

In [17]:
# while(g.latlng is None):
#     g = geocoder.google('M5G, Toronto, Ontario')


###### Since geoCoder didn't work in my context.

In [18]:
latlong = pd.read_csv('Geospatial_Coordinates.csv')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_ = pd.merge(df_clean, latlong, how='inner', left_on='PostalCode', right_on='Postal Code', right_index=False)

In [20]:
df_clean = df_[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [21]:
df_clean.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Question 3

In [22]:
import folium

## finding latitude and longitude of Toronoto
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [23]:
#selecting Borough with Toronto
TorontoData = df_clean[df_clean.Borough.str.contains('Toronto')].reset_index(drop=True)

In [24]:
##credentials
client_id = 'IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V'
client_key='YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY'
version='20200718'

In [25]:
## finding venus around different postal code
lat, long = TorontoData.iloc[0, -2:].tolist()
radius = 500
limit = 100
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_key}&v={version}&ll={lat},{long}&radius={radius}&limit={limit}'
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V&client_secret=YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY&v=20200718&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [26]:
result = requests.get(url).json()

In [27]:
result['response'].keys()

dict_keys(['suggestedFilters', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [28]:
result['response']['groups'][0]['items'][10]['venue']

{'id': '4c3e1eaa6faac9b66dc60d76',
 'name': 'Distillery Sunday Market',
 'location': {'address': '1 Trinity St',
  'lat': 43.650074989330655,
  'lng': -79.36183171318665,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.650074989330655,
    'lng': -79.36183171318665}],
  'distance': 475,
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['1 Trinity St', 'Toronto ON', 'Canada']},
 'categories': [{'id': '4bf58dd8d48988d1fa941735',
   'name': 'Farmers Market',
   'pluralName': 'Farmers Markets',
   'shortName': "Farmer's Market",
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_farmersmarket_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}

In [29]:
nearvenus = pd.json_normalize(result['response']['groups'][0]['items'])

In [30]:
nearvenus.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id',
       'venue.location.neighborhood'],
      dtype='object')

In [31]:
def extract_cat_name(row):
    try:
        cat_list = row['categories']
    except:
        cat_list = row['venue.categories']
    #print(cat_list)
    if len(cat_list) == 0:
        return None
    else:
        return cat_list[0]['name']
    

In [32]:

nearvenus.head(1).apply(extract_cat_name, axis=1)

0    Bakery
dtype: object

In [33]:
TorontoData.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


In [34]:
def venueDataFrame(names, latitude, longitude, limit=100, radius=500):
    venue_df = pd.DataFrame(columns=['Name', 'Latitude', 'Longitude', 'Categories', 'PostalCode'])
    
    for name, latitude, longitude in zip(names, latitude, longitude):
        print(f'---{name}----')
        client_id = 'IYCK1RQVEKTVPUPF5RWPJT1WUHUPAKIEBP0JBILTQORCE05V'
        client_key='YRWSSCDZT10HFC0MXTOKAHCW0TL2WPKRKHQLKJ0UWBXJ2DSY'
        version='20200718'
    
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_key}&v={version}&ll={latitude},{longitude}&radius={radius}&limit={limit}'
    
        result = requests.get(url).json()['response']['groups'][0]['items']
       
        if len(result) != 0:
            ## convert into pandas data frame
            tempdf = pd.json_normalize(result)

            #print(tempdf.columns)

            tempdf['Categories'] = tempdf.apply(extract_cat_name, axis=1)
            #print(tempdf.columns.values)
            tempdf = tempdf[['venue.name', 'venue.location.lat', 'venue.location.lng', 'Categories']]

            tempdf['Postalcode'] = name

            tempdf.columns = venue_df.columns.values

            #print(tempdf.head())

            venue_df = venue_df.append(tempdf, ignore_index=True)

        
    return venue_df
    
    

In [35]:
TorontoData.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


In [36]:
veneus_dataframe = venueDataFrame(TorontoData['PostalCode'], TorontoData['Latitude'], TorontoData['Longitude'])

---M5A----
---M7A----
---M5B----
---M5C----
---M4E----
---M5E----
---M5G----
---M6G----
---M5H----
---M6H----
---M5J----
---M6J----
---M4K----
---M5K----
---M6K----
---M4L----
---M5L----
---M4M----
---M4N----
---M5N----
---M4P----
---M5P----
---M6P----
---M4R----
---M5R----
---M6R----
---M4S----
---M5S----
---M6S----
---M4T----
---M5T----
---M4V----
---M5V----
---M4W----
---M5W----
---M4X----
---M5X----
---M4Y----
---M7Y----


In [37]:
veneus_dataframe.groupby('PostalCode').count()

,Name,Latitude,Longitude,Categories
PostalCode,,,,
M4E,4,4,4,4
M4K,42,42,42,42
M4L,22,22,22,22
M4M,42,42,42,42
M4N,4,4,4,4
M4P,9,9,9,9
M4R,20,20,20,20
M4S,35,35,35,35
M4T,3,3,3,3


In [38]:
veneus_dataframe.head()

,Name,Latitude,Longitude,Categories,PostalCode
0,Roselle Desserts,43.653447,-79.362017,Bakery,M5A
1,Tandem Coffee,43.653559,-79.361809,Coffee Shop,M5A
2,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,M5A
3,Body Blitz Spa East,43.654735,-79.359874,Spa,M5A
4,Impact Kitchen,43.656369,-79.356980,Restaurant,M5A


In [39]:
Toronto_onehot =  pd.get_dummies(veneus_dataframe.Categories, prefix='', prefix_sep='' )
Toronto_onehot['PostalCode'] = veneus_dataframe['PostalCode']
Toronto_onehot.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,PostalCode
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M5A
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M5A
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M5A
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M5A
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,M5A


In [40]:
TorontoData_Clustering = Toronto_onehot.groupby('PostalCode').mean().reset_index()
TorontoData_Clustering.head()

,PostalCode,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.25000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.0,...,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.02381
2,M4L,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.02381,0.0,0.02381
4,M4N,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


In [41]:
x = TorontoData_Clustering.drop(columns='PostalCode')
x.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.25000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.02381
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.02381,0.0,0.02381
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


In [42]:
model = KMeans(n_clusters=4, max_iter=1000, verbose=1).fit(x)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.8660596527586828
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.8660596527586828
center shift 0.000000e+00 within tolerance 4.106969e-08
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 2.6117887035654297
start iteration
done sorting
end inner loop
Iteration 1, inertia 2.6117887035654297
center shift 0.000000e+00 within tolerance 4.106969e-08
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 2.1000503986066383
start iteration
done sorting
end inner loop
Iteration 1, inertia 2.1000503986066383
center shift 0.000000e+00 within tolerance 4.106969e-08
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 2.1980396615728406
start iteration
done sorting
end inner loop
Iteration 1, inertia 2.0781473387084377
start iteration
done sorting
end inner loop
Iteration 2, inertia

In [43]:
labels = model.labels_

In [44]:
## insert columns
TorontoData_Clustering.insert(1, 'ClusterLabel', labels)
TorontoData_Clustering.head()

,PostalCode,ClusterLabel,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.25000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
1,M4K,0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,...,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.02381
2,M4L,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
3,M4M,0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.02381,0.0,0.02381
4,M4N,3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


In [45]:
MergedData = pd.merge(TorontoData, TorontoData_Clustering, how='inner', on='PostalCode')
MergedData.head()

,PostalCode,Borough,Neighborhood_x,Latitude,Longitude,ClusterLabel,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,0.022727
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.000000,0.0,0.028571
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.00,0.0,0.000000,0.01,0.01,0.010000,0.0,0.000000
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.011628,0.00,0.00,0.011628,0.0,0.000000
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.25,0.0,0.000000,0.00,0.00,0.000000,0.0,0.000000


In [46]:
geolocation = Nominatim(user_agent='Toronto')
location = geolocation.geocode('Toronto')
lat, long = location.latitude,location.longitude
lat, long

(43.6534817, -79.3839347)

In [47]:
TorontoMap = folium.Map(location=[lat, long], zoom_start=12)
#TorontoMap

In [48]:
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, 10))
rainbow = [colors.rgb2hex(i) for i in colors_array[5:9]]
colors_array[-4:]

array([[8.33333333e-01, 8.66025404e-01, 5.00000000e-01, 1.00000000e+00],
       [1.00000000e+00, 6.36474236e-01, 3.38158275e-01, 1.00000000e+00],
       [1.00000000e+00, 3.38158275e-01, 1.71625679e-01, 1.00000000e+00],
       [1.00000000e+00, 1.22464680e-16, 6.12323400e-17, 1.00000000e+00]])

In [49]:
for postalCode, lat, long, cluster in zip(MergedData['PostalCode'], MergedData['Latitude'], MergedData['Longitude'], MergedData['ClusterLabel']):
    popup = folium.Popup(postalCode+ ' '+ str(cluster), parse_html=True)
    folium.CircleMarker(location=[lat, long], 
                        radius=10, 
                        popup=popup,
                        fill=True,
                        color = rainbow[cluster-1],
                        fill_color=rainbow[cluster-1],
                        fill_opacity=0.7
                       ).add_to(TorontoMap)

In [50]:
TorontoMap